In [6]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Reshape
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adam, Adadelta
import numpy as np
import pandas as pd
from keras.utils import np_utils
import time
from keras.models import Sequential
import tempfile
from keras.models import save_model, load_model

In [2]:
train_data=pd.read_csv('train.csv')

In [3]:
train_data.head()

,label,feature
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [4]:
train_data.shape

(28709, 2)

In [5]:
pixel=train_data['feature'].values
label=train_data['label'].values

In [6]:
pixel.shape

(28709,)

In [7]:
label.shape

(28709,)

In [8]:
pixel[0]

'70 80 82 72 58 58 60 63 54 58 60 48 89 115 121 119 115 110 98 91 84 84 90 99 110 126 143 153 158 171 169 172 169 165 129 110 113 107 95 79 66 62 56 57 61 52 43 41 65 61 58 57 56 69 75 70 65 56 54 105 146 154 151 151 155 155 150 147 147 148 152 158 164 172 177 182 186 189 188 190 188 180 167 116 95 103 97 77 72 62 55 58 54 56 52 44 50 43 54 64 63 71 68 64 52 66 119 156 161 164 163 164 167 168 170 174 175 176 178 179 183 187 190 195 197 198 197 198 195 191 190 145 86 100 90 65 57 60 54 51 41 49 56 47 38 44 63 55 46 52 54 55 83 138 157 158 165 168 172 171 173 176 179 179 180 182 185 187 189 189 192 197 200 199 196 198 200 198 197 177 91 87 96 58 58 59 51 42 37 41 47 45 37 35 36 30 41 47 59 94 141 159 161 161 164 170 171 172 176 178 179 182 183 183 187 189 192 192 194 195 200 200 199 199 200 201 197 193 111 71 108 69 55 61 51 42 43 56 54 44 24 29 31 45 61 72 100 136 150 159 163 162 163 170 172 171 174 177 177 180 187 186 187 189 192 192 194 195 196 197 199 200 201 200 197 201 137 58 98 92

In [9]:
type(pixel)

numpy.ndarray

In [10]:
pixels=[ np.fromstring(x, dtype=float, sep=' ') for x in pixel ]

In [11]:
pixels[0]

array([  70.,   80.,   82., ...,  106.,  109.,   82.])

In [12]:
pixels=np.array(pixels)
pixels=pixels.reshape(pixels.shape[0],48,48,1)
pixels=pixels.astype('float32')
pixels=pixels/255

In [13]:
pixels.shape

(28709, 48, 48, 1)

In [14]:
# 将类向量转换为二进制类矩阵
label = np_utils.to_categorical(label, 7)
label.shape

(28709, 7)

In [15]:
#分成训练集和测试集
id=int(pixels.shape[0]*0.8)
X_train=pixels[:id]
X_test=pixels[id:]
y_train=label[:id]
y_test=label[id:]

In [16]:
X_train.shape

(22967, 48, 48, 1)

In [17]:
X_test.shape

(5742, 48, 48, 1)

In [18]:
y_train.shape

(22967, 7)

In [66]:
# 定义超参数
n_filter=32
batch_size = 256
nb_classes = 7  # 分类树
nb_epoch = 120  # 训练轮数

img_rows, img_cols = 48, 48  # 图片维数
kernel_size = (8, 8)  # 卷积核大小
pool_size = (6, 6)  # 最大池化，池化核大小


model=Sequential()
model.add(Conv2D(n_filter,kernel_size,strides=1,input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))

model.add(Flatten())  # Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡
model.add(Dense(689))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# 模型编译
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 41, 41, 32)        2080      
_________________________________________________________________
activation_77 (Activation)   (None, 41, 41, 32)        0         
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 1152)              0         
_________________________________________________________________
dense_52 (Dense)             (None, 689)               794417    
_________________________________________________________________
activation_78 (Activation)   (None, 689)               0         
__________

In [67]:
# 模型训练
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,validation_data=(X_test, y_test))

# 模型评估或模型预测
score = model.evaluate(X_train, y_train)
print('\nTrain score:', score[0])  # 损失值  Test score: 
print('Train accuracy:', score[1])  # 准确率  Test accuracy: 
score = model.evaluate(X_test, y_test,)
print('\nTest score:', score[0])  # 损失值  Test score: 
print('Test accuracy:', score[1])  # 准确率  Test accuracy: 


Train on 22967 samples, validate on 5742 samples
Epoch 1/120
22967/22967 [==============================] - 2s - loss: 1.8210 - acc: 0.2464 - val_loss: 1.7758 - val_acc: 0.2680
Epoch 2/120
22967/22967 [==============================] - 1s - loss: 1.7486 - acc: 0.3005 - val_loss: 1.6713 - val_acc: 0.3870
Epoch 3/120
22967/22967 [==============================] - 1s - loss: 1.6808 - acc: 0.3354 - val_loss: 1.6424 - val_acc: 0.3899
Epoch 4/120
22967/22967 [==============================] - 1s - loss: 1.6428 - acc: 0.3607 - val_loss: 1.5741 - val_acc: 0.4103
Epoch 5/120
22967/22967 [==============================] - 1s - loss: 1.6191 - acc: 0.3744 - val_loss: 1.5591 - val_acc: 0.4093
Epoch 6/120
22967/22967 [==============================] - 1s - loss: 1.6018 - acc: 0.3792 - val_loss: 1.5379 - val_acc: 0.4026
Epoch 7/120
22967/22967 [==============================] - 1s - loss: 1.5862 - acc: 0.3849 - val_loss: 1.5209 - val_acc: 0.4270
Epoch 8/120
22967/22967 [==============================

In [72]:
# 保存模型
_, fname = tempfile.mkstemp('.h5', dir='.')
save_model(model, fname)
new_model = load_model(fname)

In [5]:
fname='tmpws60mjj1.h5'
new_model = load_model(fname)
# 画模型图
from keras.utils.vis_utils import plot_model
plot_model(new_model,show_shapes=True,to_file='model.png')

In [8]:
# 用训练好的模型预测未知图片的情感
test_data=pd.read_csv('test.csv')

In [9]:
test_data.head()

,id,feature
0,0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...
1,1,156 184 198 202 204 207 210 212 213 214 215 21...
2,2,69 118 61 60 96 121 103 87 103 88 70 90 115 12...
3,3,205 203 236 157 83 158 120 116 94 86 155 180 2...
4,4,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...


In [10]:
test_data.shape

(7178, 2)

In [11]:
test_id=test_data['id'].values
test_pixel=test_data['feature'].values

In [13]:
test_pixels=[ np.fromstring(x, dtype=float, sep=' ') for x in test_pixel ]
test_pixels=np.array(test_pixels)
test_pixels=test_pixels.reshape(test_pixels.shape[0],48,48,1)
test_pixels=test_pixels.astype('float32')
test_pixels=test_pixels/255

In [14]:
test_pixels.shape

(7178, 48, 48, 1)

In [15]:
y_pred=new_model.predict(test_pixels)

In [44]:
y_pred[2][:]

array([ 0.26876205,  0.01692498,  0.13947614,  0.03221936,  0.27480349,
        0.01865733,  0.24915668], dtype=float32)

In [45]:
def find(x):
    count=0
    maxnum=0
    maxcount=0
    for i in x:
        if maxnum<i:
            maxcount=count
            maxnum=i
        count=count+1
    return maxcount
y_label=[find(x) for x in y_pred]

In [46]:
y_label=np.array(y_label)

In [47]:
y_label.shape

(7178,)

In [48]:
y_label[46]

4

In [49]:
pd.DataFrame({"id": test_id, "label": y_label}).to_csv('submission.csv',index=False)